### Problem-Independent Machinery

In [2]:
## The search algorithm itself -- it takes a problem, which will give it an initial state and the goal test,
##   the world state itself which gives it the successor states, and an evaluator that evaluates the quality
##   of a node on the search frontier

#  Priority queue code taken from Pacman project -- PriorityQueue supports
#      pop, isEmpty, and push/update
#
#  Client supplies
#    -- a WorldState; a WorldState implements the method successors()
#    -- a Problem which supplies the initial state and goal state checker
#    -- an Evaluator which supplies a method that evaluates a WorldState
#
#   The search function uses a SearchState which is a WorldState plus a sequence of 
#     actions (not examined by Search).   The search fringe is a priority 
#     queue of SearchState
#
#   Search returns a 2-tuple -- 
#    -- a sequence of actions
#    -- performance information:  process time used, number of nodes expanded, 
#         number of nodes skipped (because they were previously expanded)

from priorityqueue import PriorityQueue
import time

def aStarSearch(problem, evaluator, verbose=None):
    startTime = time.process_time()
    fringe = PriorityQueue()
    visited = {}
    initialWorldState = problem.initial()
    initialValue = evaluator.value(initialWorldState, [])
    initialSearchState = SearchState(initialWorldState, [])
    fringe.update(initialSearchState, initialValue)
    numVisited = numSkipped = 0
    while (True):
        if fringe.isEmpty():
            return (None, (time.process_time() - startTime, numVisited, numSkipped))
        nextNode = fringe.pop()   # A search state (state, actions)
        numVisited += 1
        if (verbose and numVisited % verbose == 0):
            print("Visited " + str(numVisited) + " world is " + str(nextNode._worldState))
            print("Skipped " + str(numSkipped) + " Fringe is size " + str(len(fringe.heap)))
            print("Evaluation is " + str(evaluator.value(nextNode._worldState, nextNode._actions)) + " with actions " + str(len(nextNode._actions)))
        if (problem.isGoal(nextNode.worldState())):
            return (nextNode._actions, (time.process_time() - startTime, numVisited, numSkipped))
        if (nextNode._worldState in visited):
            numSkipped += 1
        else:
            visited[nextNode.worldState()] = True
            successors = nextNode.worldState().successors()
            for successor in successors:
                state, action = successor
                actions = list(nextNode.actions())
                actions.append(action)
                newSS = SearchState(state, actions)
                newValue = evaluator.value(state, actions)
                fringe.update(newSS, newValue)
    raise "Impossible search execution path."

## Instances of SearchState go on the search fringe -- contains both a state and 
## list of actions so far

class SearchState:
    def __str__(self):
        return "{S " + str(self._worldState) + "/" + str(self._actions) + "}"
    
    def __init__(self, worldState, actions):
        self._worldState = worldState
        self._actions = actions
    
    def worldState(self):
        return self._worldState
    
    def actions(self):
        return self._actions

### Interface Between Search and Client ###

In [3]:
class WorldState:
    #  Method successors() returns a tuple:  (worldState, action)
    def successors():
        raise "Not implemented"

# Evaluator provides the evaluator f(s) = g(s) + h*(s)

class Evaluator:
    def __init__(self, goalEstimator, actionsCoster):
        self._estimator = goalEstimator
        self._coster = actionsCoster
    def estimateToGoal(self, state):
        return self._estimator(state)
    def costSoFar(self, actions):
        return self._coster(actions)
    def value(self, state, actions):
        return self.estimateToGoal(state) + self.costSoFar(actions)
    

In [4]:
import copy

class PancakeWorldState(WorldState):

    def __init__(self, arr):
        self._list = arr
        
    def __str__(self):
        return str(self._list)
    
    #  Any world state needs to redefine equality and hash so instances go on the priority queue correctly
    #  and the duplication check works
    
    def __eq__(self, other):
        if isinstance(other, PancakeWorldState):
            return self._list == other._list
        else:
            return False

    def __hash__(self):
        return hash(str(self._list) + str(len(self._list)))
    
    
    def successors(self):
        candidates = [self.flip(x) for x in range(1, len(self._list))]
        return candidates
    
    def flip(self, index):
        inputList = self._list
        newList = flipArray(inputList, index)
        newS = PancakeWorldState(newList)   
        return (newS, "flip_" + str(index))
    
    def isGoal(self):
        return arrayIsSorted(self._list)

def flipArray(array, flip):
    array = copy.copy(array)
    return array[:flip+1][::-1] + array[flip+1:]

def arrayIsSorted(array):
    for i in range(0,len(array)-1):
        if (array[i] > array[i+1]):
            return False
    return True


In [5]:
class PancakeWorldProblem:
    def __init__(self, startList):
        self._initial = startList
  
    def initial(self):
        return(PancakeWorldState(self._initial))
    
    def isGoal(self, state):
        return state.isGoal()

In [6]:
def uniformCostCoster(actions):
    return len(actions)

In [10]:
def pancakeSort(startList):
    testPancakeProblem = PancakeWorldProblem(startList)
    bfsEvaluator = Evaluator(lambda state: 0, uniformCostCoster)
    actions, stats = aStarSearch(testPancakeProblem, bfsEvaluator)
    resultString = "Expanded " + str(stats[1]) + "\n" + str(actions)
    print("Stats: " + str(stats))
    return resultString

In [8]:
print(pancakeSort([3,1,4,5,2]))

Stats: (0.0, 90, 27)
Expanded 90
['flip_1', 'flip_3', 'flip_4', 'flip_1']


In [11]:
print(pancakeSort([]))
print(pancakeSort([0]))
print(pancakeSort([0,1]))
print(pancakeSort([3,2,4,5,1,7,6]))


Stats: (0.0, 1, 0)
Expanded 1
[]
Stats: (0.0, 1, 0)
Expanded 1
[]
Stats: (0.0, 1, 0)
Expanded 1
[]
Stats: (0.671875, 1356, 502)
Expanded 1356
['flip_1', 'flip_3', 'flip_5', 'flip_6', 'flip_5']
